In [ ]:
#IAA Room Reservation 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
from threading import Timer
import schedule
import time

def book_a_room():

    options = webdriver.ChromeOptions()
    options.add_argument("Chrome_Path") #Path to chrome profile

    driver_path = 'Driver_Path'
    
    #ONLY ONE OF THE FOLLOWING SHOULD EXIST
    browser = webdriver.Chrome(executable_path=driver_path, options=options)
    #browser = webdriver.Chrome(driver_path)
    
    browser.get("URL")
    
    #---------SHIBBOLETH-------
    
    #Fill in username and password
    email_elem = browser.find_element_by_id("username").send_keys('username')
    password_elem = browser.find_element_by_id("password").send_keys('password')
    
    #Click log in button
    LogIn_Button = browser.find_element_by_xpath("//*[@id='formSubmit']").click()
    
    #At this point, the program successfully logs into the Shibboleth service. A duo push is required to advance.
    #I'll have it send me a push...it's not full automation but the push serves as a daily reminder and an instant
    #method for room reservation
    
    #---------DUO-------
    
    #Click 'send push'
    ## Give time for iframe to load ##
    time.sleep(3)
    ## You have to switch to the iframe like so: ##
    browser.switch_to.frame(browser.find_element_by_tag_name("iframe"))
    ## Insert text via xpath ##
    Push_Button = browser.find_element_by_xpath("//*[@id='auth_methods']/fieldset/div[1]/button").click()
    ## Switch back to the "default content" (that is, out of the iframes) ##
    browser.switch_to.default_content()
    
    #-------Click accept (#1) to proceed
    wait_accept1 = WebDriverWait(browser, 10)
    element_accept1 = wait_accept1.until(EC.element_to_be_clickable((By.XPATH,
                                                                   "//*[@id='content']/main/form/div[1]/p[2]/input[2]")))
    Wait_button1 = browser.find_element_by_xpath("//*[@id='content']/main/form/div[1]/p[2]/input[2]").click()
    
    #--------Click "IAA Conference Room Scheduler"
    wait_IAA_conf = WebDriverWait(browser,10)
    element_IAA_conf = wait_IAA_conf.until(EC.element_to_be_clickable((By.XPATH,"//*[@id='ui-id-14']/p/b/a")))
    Iaa_Room_Button = browser.find_element_by_xpath("//*[@id='ui-id-14']/p/b/a").click()
    
    #-------Click accept (#2) to proceed
    #First, switch to the new tab
    time.sleep(5)
    browser.switch_to.window(browser.window_handles[1])
    wait_accept2 = WebDriverWait(browser, 10)
    element_accept2 = wait_accept2.until(EC.element_to_be_clickable((By.XPATH,
                                                                 "//*[@id='content']/main/form/div[1]/p[2]/input[2]")))
    Wait_button2 = browser.find_element_by_xpath("//*[@id='content']/main/form/div[1]/p[2]/input[2]").click()

    
    #-----Reserve room 5 weeks in advance-------
    
    #1) Enter 'option + N' m times to advance m days in calendar (35 for 5 weeks)
    m = 42
    i = 1
    while i <= m:
        browser.find_element_by_tag_name('body').send_keys(u'\ue00a','n')
        i = i + 1
    
    #2) Reserve room from 12:00 - 12:30 PM
    wait_room = WebDriverWait(browser,10)
    element_room = wait_room.until(EC.element_to_be_clickable((By.XPATH,"//*[@id='onslow-1200']")))
    room_button = browser.find_element_by_xpath("//*[@id='onslow-1200']").click()
    browser.find_element_by_xpath("//*[@id='onslow-1230']").click()
    
    #Add delay so that the program runs every day (every 86,400 seconds)
    x=datetime.today()
    y=x.replace(day=x.day, hour=x.minute, minute=x.second, second=x.second, microsecond=x.microsecond) + timedelta(minutes = 1)
    delta = y - x
    secs=delta.total_seconds()
    time.sleep(secs)
    
    
    

In [ ]:
import schedule
import time

schedule.every().day.at("21:00").do(book_a_room)

while False:
    schedule.run_pending()
    time.sleep(1)